In [1]:
import cv2
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd

# CATEGORIES = ["boxing", "handclapping", "handwaving", "jogging", "running", 
#     "walking"]


In [2]:
#TRAIN_PEOPLE_ID = [11, 12, 13, 14, 15, 16, 17, 18]
#DEV_PEOPLE_ID = [19, 20, 21, 23, 24, 25, 1, 4]
#TEST_PEOPLE_ID = [22, 2, 3, 5, 6, 7, 8, 9, 10]
TRAIN_PEOPLE_ID = [1, 2, 3, 4]
TEST_PEOPLE_ID = [5]

In [3]:
data_gen_path = "../data_generated"
#os.listdir(data_gen_path)
no_cuboids = 48
feature_lenght = 192

def concatenate_features(path):
    files = os.listdir(path)
    angles = []
    magnitudes = []
    
    
    
    train = []
    test = []
    print('Nº files:', len(files))
    
    for file in files:
        person_id = int(file.split("_")[2][7:])
#        print(person_id)
        features = []
        pt = os.path.join(data_gen_path,file)
        #print('processing video_csv: ',pt)
        df = pd.read_csv(pt)
        for i in range(no_cuboids): #for each row
            f_angles = list(df.iloc[i,0:feature_lenght])#first 192 columns
            f_magnitudes = list(df.iloc[i,feature_lenght:2*feature_lenght])  #last 192 columns
            angles.append(f_angles)
            magnitudes.append(f_magnitudes)
            
            features.append(np.array(f_magnitudes))
        
        if person_id in TRAIN_PEOPLE_ID:
            train.append({
                "video": file,
                "category": df.activity[0], 
                "features": features
            })
        else: test.append({
                "video": file,
                "category": df.activity[0], 
                "features": features
            })
    
    print('Nº angles:', len(angles))
    print('Nº magnitudes:', len(magnitudes))
    print('train len:', len(train))
    print('test len:', len(test))
    return train, test, angles, magnitudes

train, test ,angles,magnitudes = concatenate_features(data_gen_path)
pickle.dump(train, open("../pickle/train.p", "wb"))
pickle.dump(test, open("../pickle/test.p", "wb"))


Nº files: 120
Nº angles: 5760
Nº magnitudes: 5760
train len: 96
test len: 24


In [ ]:
train[0]["features"]

In [4]:
clusters = 100

from sklearn.cluster import KMeans
from numpy import size

# Clustering with KMeans.
print("Running KMeans clustering")
kmeans = KMeans(init='k-means++', n_clusters=clusters, n_init=10, n_jobs=2,
    verbose=1)
kmeans.fit(angles)
# Save trained kmeans object to file.
pickle.dump(kmeans, open("../pickle/cb_%dclusters.p" % clusters, "wb"))

Running KMeans clustering
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 1.2002805695775834e+23
start iteration
done sorting
end inner loop
Iteration 1, inertia 1.1621826661906097e+23
start iteration
done sorting
end inner loop
Iteration 2, inertia 1.1510676720237421e+23
start iteration
done sorting
end inner loop
Iteration 3, inertia 1.1498634539712135e+23
start iteration
done sorting
end inner loop
Iteration 4, inertia 1.1498634539712135e+23
center shift 0.000000e+00 within tolerance 4.809929e+14
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 1.2321709821429062e+23
start iteration
done sorting
end inner loop
Iteration 1, inertia 1.190626350449545e+23
start iteration
done sorting
end inner loop
Iteration 2, inertia 1.1778327616211698e+23
start iteration
done sorting
end inner loop
Iteration 3, inertia 1.1778327616211698e+23
center shift 0.000000e+00 within tolerance 4.809929e+14
Initialization complete

## Building Bag of Words

In [5]:
from scipy.cluster.vq import vq

def make_bow(dataset, clusters):
    n_videos = len(dataset)
    bow = np.zeros((n_videos, clusters.shape[0]), dtype=np.float)
    # Make bow vectors for all videos.
    video_index = 0
    for video in dataset:
        visual_word_ids = vq(video["features"], clusters)[0]
        #print(len(visual_word_ids))
        for word_id in visual_word_ids:
            bow[video_index, word_id] += 1
        video_index += 1
        
    # Check whether to use TF-IDF weighting.
    tfidf = False
    if tfidf:
        print("Applying TF-IDF weighting")
        freq = np.sum((bow > 0) * 1, axis = 0)
        idf = np.log((n_videos + 1) / (freq + 1))
        bow = bow * idf

    # Replace features in dataset with the bow vector we've computed.
    video_index = 0
    for i in range(len(dataset)): #for each video
        dataset[i]["features"] = bow[video_index]
        video_index += 1

        #if (i + 1) % 50 == 0:
        #    print("Processed %d/%d videos" % (i + 1, len(dataset)))
    print("reeplaced features by bow...")
    return dataset


dataset_train  = "../pickle/train.p"
dataset_test  = "../pickle/test.p"
codebook_file = "../pickle/cb_100clusters.p"
# Load clusters.
codebook = pickle.load(open(codebook_file, "rb"))
clusters = codebook.cluster_centers_

# Load dataset.
dataset_train = pickle.load(open(dataset_train, "rb"))
dataset_test = pickle.load(open(dataset_test, "rb"))

train_bow = make_bow(dataset_train, clusters)
test_bow = make_bow(dataset_test, clusters)
#len(test_bow), train_bow

reeplaced features by bow...
reeplaced features by bow...


In [6]:
clusters[0].shape, len(dataset_train[0]["features"])

((192,), 100)

In [7]:
dataset_train[0]["features"].shape

(100,)

## Train

In [8]:
from sklearn.svm import SVC

def make_dataset(dataset):
    X = []
    Y = []

    for video in dataset:
        X.append(video["features"])
        Y.append(video["category"])
    return X,Y
        
X_train, Y_train = make_dataset(dataset_train)
X_test, Y_test = make_dataset(dataset_test)

In [9]:
X_test

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., 46.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  2.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  

In [10]:
# Train SVM and save to file.
clf = SVC(C=1, kernel="linear", verbose=True)
clf.fit(X_train, Y_train)

[LibSVM]

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=True)

## Prediction

In [11]:
correct = 0
for video in dataset_train:    
    #video = dataset[0]
    predicted = clf.predict([video["features"]])
    print(predicted)
    # Check if majority is correct.
    if predicted == video["category"]:
        correct += 1
print("%d/%d Correct" % (correct, len(dataset_train)))
print("Accuracy =", correct / len(dataset_train))

['running']
['handclapping']
['handclapping']
['running']
['jogging']
['handclapping']
['running']
['handwaving']
['running']
['running']
['running']
['running']
['jogging']
['handwaving']
['handwaving']
['jogging']
['walking']
['jogging']
['handclapping']
['walking']
['running']
['handclapping']
['boxing']
['walking']
['handclapping']
['handclapping']
['boxing']
['running']
['boxing']
['running']
['jogging']
['handwaving']
['running']
['walking']
['running']
['boxing']
['running']
['handwaving']
['running']
['walking']
['jogging']
['handwaving']
['boxing']
['running']
['walking']
['running']
['running']
['boxing']
['walking']
['handclapping']
['boxing']
['handclapping']
['running']
['walking']
['handclapping']
['handclapping']
['boxing']
['walking']
['handclapping']
['jogging']
['handclapping']
['walking']
['handclapping']
['running']
['jogging']
['handwaving']
['running']
['walking']
['running']
['running']
['running']
['running']
['jogging']
['boxing']
['running']
['running']
['boxi

In [ ]:
Y_test

In [ ]:
Y

## train

In [ ]:
from sklearn.svm import SVC

def make_dataset(dataset):
    X = []
    Y = []

    for video in dataset:
        X.append(video["features"])
        Y.append(video["category"])
    return X,Y
        
X, Y = make_dataset(dataset)


In [ ]:
X

In [ ]:
# Train SVM and save to file.
clf = SVC(C=1, kernel="linear", verbose=True)
clf.fit(X, Y)

In [ ]:
X_test = dataset[1]['features']
Y_test = dataset[1]['category']

In [ ]:
X_test = X
Y_test = Y

In [ ]:
Y_pred = clf.predict(X)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_pred, Y_test)